# Dados de Logística - Analisando performance e SLA de transportadoras 

## Introdução
Neste notebook procurei realizar uma análise exploratória de um conjunto de dados logísticos fictícios, focando em medir o desempenho de SLA (Service Level Agreement) e OTD (On-Time Delivery). As métricas e visualizações foram desenvolvidas para apresentar padrões e insights úteis para a gestão de transporte e logística aos leads interessados.

---

## Importando as Libs do notebook

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## Configuração inicial das visualizações dos dados

In [2]:
sns.set(style='whitegrid')


## Carregando os Dados

In [3]:
df = pd.read_csv('csv/dados_logisticos_5000.csv', sep=";")

In [ ]:

# Validando se os dados foram carregados corretamente

validate = df.head()

print(validate)

## Limpeza e Transformação dos Dados
As colunas de datas são convertidas para o formato datetime, e colunas adicionais são criadas para facilitar a análise:

- **Dias de Atraso**: Diferença entre a data de entrega real e a data prevista.
- **Cumpriu SLA**: Indicador binário (1 para entrega dentro do prazo, 0 caso contrário).

In [5]:
# Converter colunas de datas para datetime
df["Data de Pedido"] = pd.to_datetime(df["Data de Pedido"])
df["Data Prevista de Entrega"] = pd.to_datetime(df["Data Prevista de Entrega"])
df["Data de Entrega"] = pd.to_datetime(df["Data de Entrega"], errors='coerce')

# Criar coluna "Dias de Atraso"
df["Dias de Atraso"] = (df["Data de Entrega"] - df["Data Prevista de Entrega"]).dt.days

# Criar coluna "Cumpriu SLA"
df["Cumpriu SLA"] = df["Dias de Atraso"].apply(lambda x: 1 if pd.notnull(x) and x <= 0 else 0)

In [ ]:
# Validando as colunas
print(df.head())

## Métricas Gerais

- Percentual geral de SLA cumprido.
- Contagem de status de entrega.
- Percentual de SLA por transportadora.

In [7]:
# Percentual de SLA
sla_percentual = df['Cumpriu SLA'].mean() * 100
status_cont = df['Status'].value_counts()
sla_transp = df.groupby('Transportadora')['Cumpriu SLA'].mean() * 100

In [ ]:
print(sla_percentual)

In [ ]:
print(status_cont)

In [ ]:
print(sla_transp)

---

## Análise das métricas com Dashboards

### 1. Percentual de SLA por Transportadora
- Gráfico de Barras:

In [ ]:
plt.figure(figsize=(10, 6))
sla_transp.sort_values().plot(kind='bar', color='skyblue')
plt.title('Percentual de SLA por Transportadora')
plt.ylabel('SLA %')
plt.xlabel('Transportadora')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 2. Contagem de Status de Entrega
- Gráfico de Barras

In [ ]:
plt.figure(figsize=(10, 6))
status_cont.plot(kind='bar', color='orange')
plt.title('Contagem de Status de Entrega')
plt.ylabel('Quantidade')
plt.xlabel('Status')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### 3. Atraso de Entrega x Distância em KM
- Gráfico de Dispersão:

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatterplot(data=df, x='Distância (km)', y='Dias de Atraso', hue='Transportadora')
plt.title('Atraso de Entrega vs Distância em KM')
plt.ylabel('Dias de Atraso')
plt.xlabel('Distância (KM)')
plt.tight_layout()
plt.show()